### 기존 코드와 Gemini연결

In [1]:
import pandas as pd
import json
import google.generativeai as genai

### 데이터 전처리1
코드와 정보값을 구분하여 정리

In [2]:
# 엑셀 파일을 읽어옵니다
code_df = pd.read_excel("../data/코드정의서_base.xlsx")  # 파일 경로를 지정하세요

# 데이터프레임을 리스트로 변환합니다
data_list = code_df.values.tolist()

# code_val_dic에 키-값 형태로 코드와 정보를 저장
code_val_dic = {}
val_code_dic = {}

for key, value in data_list:
    code_val_dic[key] = value
for key, value in data_list:
    val_code_dic[value] = key

# print(code_val_dic)
# print(val_code_dic)
code_val_dic

{'Q1_1': '전시회 관람(미술, 사진, 건축, 디자인 등)',
 'Q1_2': '박물관 관람',
 'Q1_3': '음악연주회 관람(클래식, 오페라 등)',
 'Q1_4': '전통예술공연 관람(국악, 민속놀이 등)',
 'Q1_5': '연극공연 관람(뮤지컬 포함)',
 'Q1_6': '무용공연 관람',
 'Q1_7': '영화관람',
 'Q1_8': '연예공연 관람(쇼, 콘서트, 마술 쇼 등)',
 'Q1_9': '문학행사참여',
 'Q1_10': '글짓기/독서토론',
 'Q1_11': '미술활동(그림, 서예, 조각, 디자인, 도예, 만화 등)',
 'Q1_12': '악기연주/노래교실',
 'Q1_13': '전통예술 배우기(사물놀이, 줄타기 등)',
 'Q1_14': '사진촬영(디지털카메라 포함)',
 'Q1_15': '춤/무용(발레, 한국무용, 현대무용, 방송댄스, 스트릿댄스, 비보잉 등)',
 'Q1_16': '스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)',
 'Q1_17': '스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)',
 'Q1_18': '격투 스포츠 경기관람(태권도, 유도, 합기도, 검도,권투 등)',
 'Q1_19': '온라인게임 경기관람(e-스포츠 경기 포함)',
 'Q1_20': '농구, 배구, 야구, 축구, 족구',
 'Q1_21': '테니스, 스쿼시',
 'Q1_22': '당구/포켓볼',
 'Q1_23': '볼링, 탁구',
 'Q1_24': '골프',
 'Q1_25': '수영',
 'Q1_26': '윈드서핑, 수상스키, 스킨스쿠버다이빙, 래프팅, 요트',
 'Q1_27': '스노보드, 스키 등',
 'Q1_28': '아이스스케이트, 아이스하키 등',
 'Q1_29': '헬스(보디빌딩)/에어로빅',
 'Q1_30': '요가/필라테스/태보',
 'Q1_31': '배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프',
 'Q1_32': '육상/조깅/속보',
 'Q1_33': '격투 스포츠(태

### 데이터 전처리2
base자료와 지난 1년간 가장 많이 참여한 여가활동데이터프레임을 분리  
분리한 데이터에도 ID부여

In [3]:
# 파일 읽어오기 - 에러발생시 현재 파일과 상대파일 경로 꼭! 재확인!
db1_df = pd.read_excel("../data/지난 1년간 가장 많이 참여한 여가활동.xlsx")

# 우리가 가진 데이터에서 base값은 91번컬럼까지있으니 확인!
base_df = db1_df.iloc[:,:92]
culture_df = db1_df.iloc[:,92:]

display(base_df.head())
display(culture_df.head())

,ID,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q1_6,Q1_7,Q1_8,Q1_9,...,Q1_82,Q1_83,Q1_84,Q1_85,Q1_86,Q1_87,Q1_88,Q1_89,Q1_90,Q1_91
0,1001005.0,0,0,0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001008.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001010.0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001018.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001025.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Q2_1_1,Q2_1_2,Q2_1_3,Q2_1_4,Q2_1_5
0,25.0,81.0,71.0,53.0,44.0
1,74.0,2.0,2.0,3.0,1.0
2,58.0,73.0,7.0,1.0,1.0
3,77.0,74.0,51.0,31.0,1.0
4,72.0,52.0,2.0,2.0,3.0


In [4]:
# culture_df에도 ID(유저) 추가 - 추후 맵핑위해서!
culture_df['ID'] = db1_df['ID']

culture_df = culture_df[['ID'] + culture_df.columns[:-1].tolist()]

culture_df

,ID,Q2_1_1,Q2_1_2,Q2_1_3,Q2_1_4,Q2_1_5
0,1001005.0,25.0,81.0,71.0,53.0,44.0
1,1001008.0,74.0,2.0,2.0,3.0,1.0
2,1001010.0,58.0,73.0,7.0,1.0,1.0
3,1001018.0,77.0,74.0,51.0,31.0,1.0
4,1001025.0,72.0,52.0,2.0,2.0,3.0
...,...,...,...,...,...,...
9662,2477060.0,71.0,75.0,58.0,32.0,63.0
9663,2477061.0,75.0,74.0,63.0,83.0,64.0
9664,2477062.0,71.0,14.0,48.0,16.0,20.0
9665,2477063.0,88.0,52.0,71.0,75.0,55.0


### 데이터전처리3
여가활동코드 딕셔너리에 저장

In [5]:
# 엑셀 파일을 읽어옵니다
code_culture_df = pd.read_excel("../data/여가유형.xlsx")  # 파일 경로를 지정하세요

# 데이터프레임을 리스트로 변환합니다
culture_data_list = code_culture_df.values.tolist()

# code_val_dic에 키-값 형태로 코드와 정보를 저장
culture_code_val_dic = {}

for key, value in culture_data_list:
    culture_code_val_dic[key] = value

culture_code_val_dic

{1: ' 전시회 관람(미술, 사진, 건축, 디자인 등)',
 2: ' 박물관 관람',
 3: ' 음악연주회 관람(클래식, 오페라 등)',
 4: ' 전통예술공연 관람(국악, 민속놀이 등)',
 5: ' 연극공연 관람(뮤지컬 포함)',
 6: ' 무용공연 관람',
 7: ' 영화관람',
 8: ' 연예공연 관람(쇼, 콘서트, 마술 쇼 등)',
 9: ' 문학행사참여',
 10: ' 글짓기/독서토론',
 11: ' 미술활동(그림, 서예, 조각, 디자인, 도예, 만화 등)',
 12: ' 악기연주/노래교실',
 13: ' 전통예술 배우기(사물놀이, 줄타기 등)',
 14: ' 사진촬영(디지털카메라 포함)',
 15: ' 춤/무용(발레, 한국무용, 현대무용, 방송댄스, 스트릿댄스, 비보잉 등)',
 16: ' 스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)',
 17: ' 스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)',
 18: ' 격투 스포츠 경기관람(태권도, 유도, 합기도, 검도,권투 등)',
 19: ' 온라인게임 경기관람(e-스포츠 경기 포함)',
 20: ' 농구, 배구, 야구, 축구, 족구',
 21: ' 테니스, 스쿼시',
 22: ' 당구/포켓볼',
 23: ' 볼링, 탁구',
 24: ' 골프',
 25: ' 수영',
 26: ' 윈드서핑, 수상스키, 스킨스쿠버다이빙, 래프팅, 요트',
 27: ' 스노보드, 스키 등',
 28: ' 아이스스케이트, 아이스하키 등',
 29: ' 헬스(보디빌딩)/에어로빅',
 30: ' 요가/필라테스/태보',
 31: ' 배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프',
 32: ' 육상/조깅/속보',
 33: ' 격투 스포츠(태권도, 유도, 합기도, 검도, 권투 등)',
 34: ' 댄스스포츠(탱고, 왈츠, 자이브, 맘보,폴카, 차차차등)',
 35: ' 사이클링/산악자전거',
 36: ' 인라인스케이트',
 37: ' 승마, 암벽등반, 철인삼종경기, 서바이벌

### TEST

Gemini모델을 활용해서 사용자의 질문을 우리가 원하는 코드값으로 반환을 받고 해당코드값으로 유사한 유저를 추출하여  
선호하는 여가활동을 반환한다.

In [6]:
# 사용자가 입력한 문장
message = input("질문을 입력하세요.")
prompt = f'''너는 자연어처리를 이용해서 코드를 반환하는 챗봇이야 아래는 키워드와 코드 딕셔너리이고 다음 사용자 질문에 비슷한 코드를 찾아서 코드로만 반환해줘 만약 비슷한 코드가 여러개라면 csv로 반환해줘

사용자:
{message}
{val_code_dic}
'''

api_key_path = 'D:/important/APIkey.json'

# API 키 및 모델 설정
def load_api_key():
    try:
        with open(api_key_path, 'r') as file:
            data = json.load(file)
            print("API 키 로딩이 완료되었습니다.")
            return data.get('Gemini')
    except FileNotFoundError:
        print("API 키 파일을 찾을 수 없습니다.")
        return None
    except json.JSONDecodeError:
        print("API 키 파일의 JSON 형식이 올바르지 않습니다.")
        return None
    
# 모델 생성부 -----------------------------------------
api_key = load_api_key()
if api_key:
    # Gemini 모델 생성
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        print("Gemini 로딩이 완료되었습니다.")
    except Exception as e:
        print(f"Gemini 모델 생성 오류: {e}")
        
response = model.generate_content(prompt)
answer = response.candidates[0].content.parts[0].text
print(answer)

API 키 로딩이 완료되었습니다.
Gemini 로딩이 완료되었습니다.
```csv
Q1_25,Q1_20,Q1_5
```


In [9]:
answer.replace('\n','').replace(' ','').replace('`','').split(',')

['csvQ1_25', 'Q1_20', 'Q1_5']

In [10]:
# 새로운 행을 생성하여 지정된 컬럼에만 1을, 나머지 컬럼에는 0을 넣음
new_row = {col: 1 if col in answer else 0 for col in db1_df.columns}
new_row['ID'] = 9999999  # ID는 None 또는 원하는 ID 값으로 설정

# DataFrame 형식으로 변환하여 기존 데이터프레임과 병합
new_row_df = pd.DataFrame([new_row])
new_df = pd.concat([db1_df, new_row_df], ignore_index=True)

new_df

,ID,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q1_6,Q1_7,Q1_8,Q1_9,...,Q1_87,Q1_88,Q1_89,Q1_90,Q1_91,Q2_1_1,Q2_1_2,Q2_1_3,Q2_1_4,Q2_1_5
0,1001005.0,0,0,0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,25.0,81.0,71.0,53.0,44.0
1,1001008.0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,74.0,2.0,2.0,3.0,1.0
2,1001010.0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,58.0,73.0,7.0,1.0,1.0
3,1001018.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,77.0,74.0,51.0,31.0,1.0
4,1001025.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,72.0,52.0,2.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,2477061.0,0,0,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,75.0,74.0,63.0,83.0,64.0
9664,2477062.0,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,71.0,14.0,48.0,16.0,20.0
9665,2477063.0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,88.0,52.0,71.0,75.0,55.0
9666,2477064.0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,84.0,71.0,72.0,55.0,86.0


In [11]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm

# 코사인 유사도 함수 정의
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

# 데이터 로드 및 3000개 샘플링 (예시)
data_vectors = db1_df.select_dtypes(include=[np.number]).iloc[:3000].values  # 숫자형 데이터만 사용
data_ids = db1_df['ID'][:3000].values  # 3000개의 ID 추출

# data_vectors의 벡터 길이에 맞춘 테스트 벡터 생성
vector_length = data_vectors.shape[1]
user_input_vector = np.random.randint(2, size=vector_length)

print("테스트 벡터:", new_df.iloc[-1,:])

# 입력 벡터와 각 데이터 벡터 간의 유사도를 계산하여 리스트에 저장
similarities = []
for i in range(len(data_vectors)):
    similarity = cos_sim(user_input_vector, data_vectors[i])
    similarities.append((data_ids[i], similarity))  # (ID, 유사도) 형태로 저장

# 유사도에 따라 내림차순 정렬
similarities.sort(key=lambda x: x[1], reverse=True)

# 상위 N개의 유사한 문서 추출 (예: 상위 5개)
top_n = 5
top_n_similarities = similarities[:top_n]

user_culture = {}

# 출력
print("사용자 입력과 가장 유사한 상위 5개 문서:")
for doc_id, sim in top_n_similarities:
    print(f"문서 ID: {doc_id}, 유사도: {sim}")
    user_culture[doc_id] = []
    row_index = 0  # 원하는 행의 인덱스 설정

    # 값이 1인 열 이름 추출
    columns_with_ones = db1_df.columns[db1_df[db1_df['ID'] == doc_id].iloc[row_index] == 1].tolist()

    for column in columns_with_ones:
        try:
            # code_val_dic에서 column 키가 존재하는 경우에만 추가
            user_culture[doc_id].append(code_val_dic[column])
        except KeyError:
            # code_val_dic에 키가 없는 경우 예외 처리
            print(f"Warning: {column} 키가 code_val_dic에 존재하지 않습니다.")
            # 필요한 경우 다른 대체값을 추가할 수 있습니다.
            user_culture[doc_id].append(None)  # None으로 빈 값 추가

print(user_culture)


테스트 벡터: ID        9999999.0
Q1_1            0.0
Q1_2            1.0
Q1_3            0.0
Q1_4            0.0
            ...    
Q2_1_1          0.0
Q2_1_2          0.0
Q2_1_3          0.0
Q2_1_4          0.0
Q2_1_5          0.0
Name: 9667, Length: 97, dtype: float64
사용자 입력과 가장 유사한 상위 5개 문서:
문서 ID: 1027057.0, 유사도: 0.13611615105235242
문서 ID: 1027064.0, 유사도: 0.13611615082487527
문서 ID: 1046081.0, 유사도: 0.13611606460345407
문서 ID: 1026017.0, 유사도: 0.13611591946745752
문서 ID: 1055006.0, 유사도: 0.13611578248589729
{1027057.0: ['영화관람', '테마파크가기/놀이공원/동물원/식물원 가기', '노래방 가기', '게임(온라인/모바일/콘솔게임 등)', '쇼핑/외식', '음주', 'TV시청(IPTV 포함)', '모바일 컨텐츠, 동영상, VOD 시청, OTT 시청', '음악 감상', '잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)', '친구만남/이성교제/미팅/소개팅'], 1027064.0: ['영화관람', '테마파크가기/놀이공원/동물원/식물원 가기', '노래방 가기', '게임(온라인/모바일/콘솔게임 등)', '쇼핑/외식', 'TV시청(IPTV 포함)', '모바일 컨텐츠, 동영상, VOD 시청, OTT 시청', '음악 감상', '잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)', '친구만남/이성교제/미팅/소개팅'], 1046081.0: ['연극공연 관람(뮤지컬 포함)', '영화관람', '사진촬영(디지털카메라 포함)', '스포츠 경기 간접관람- TV, DMB를 통

In [12]:
code_list = ['Q2_1_1', 'Q2_1_2', 'Q2_1_3', 'Q2_1_4', 'Q2_1_5']
document = ''

for i in user_culture.keys():
    print(f'userID:{i}')
    for idx, code in enumerate(code_list):
        value = culture_df.loc[culture_df['ID'] == i, code].values[0]
        print(f'{idx+1}순위 : {culture_code_val_dic[value]}')
        document += f'{idx+1}순위 : {culture_code_val_dic[value]} '
    print('-'*50)

userID:1027057.0
1순위 :  모바일 컨텐츠, 동영상, VOD 시청, OTT 시청
2순위 :  TV시청(IPTV 포함)
3순위 :  친구만남/이성교제/미팅/소개팅
4순위 :  게임(온라인/모바일/콘솔게임 등)
5순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)
--------------------------------------------------
userID:1027064.0
1순위 :  모바일 컨텐츠, 동영상, VOD 시청, OTT 시청
2순위 :  TV시청(IPTV 포함)
3순위 :  친구만남/이성교제/미팅/소개팅
4순위 :  게임(온라인/모바일/콘솔게임 등)
5순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)
--------------------------------------------------
userID:1046081.0
1순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)
2순위 :  인터넷 검색/1인 미디어 제작/SNS/영상편집
3순위 :  친구만남/이성교제/미팅/소개팅
4순위 :  볼링, 탁구
5순위 :  산책 및 걷기
--------------------------------------------------
userID:1026017.0
1순위 :  TV시청(IPTV 포함)
2순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)
3순위 :  친구만남/이성교제/미팅/소개팅
4순위 :  산책 및 걷기
5순위 :  종교활동
--------------------------------------------------
userID:1055006.0
1순위 :  TV시청(IPTV 포함)
2순위 :  산책 및 걷기
3순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)
4순위 :  목욕/사우나/찜질방
5순위 :  친구만남/이성교제/미팅/소개팅
--------------------------------------------------


In [13]:
document

'1순위 :  모바일 컨텐츠, 동영상, VOD 시청, OTT 시청 2순위 :  TV시청(IPTV 포함) 3순위 :  친구만남/이성교제/미팅/소개팅 4순위 :  게임(온라인/모바일/콘솔게임 등) 5순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함) 1순위 :  모바일 컨텐츠, 동영상, VOD 시청, OTT 시청 2순위 :  TV시청(IPTV 포함) 3순위 :  친구만남/이성교제/미팅/소개팅 4순위 :  게임(온라인/모바일/콘솔게임 등) 5순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함) 1순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함) 2순위 :  인터넷 검색/1인 미디어 제작/SNS/영상편집 3순위 :  친구만남/이성교제/미팅/소개팅 4순위 :  볼링, 탁구 5순위 :  산책 및 걷기 1순위 :  TV시청(IPTV 포함) 2순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함) 3순위 :  친구만남/이성교제/미팅/소개팅 4순위 :  산책 및 걷기 5순위 :  종교활동 1순위 :  TV시청(IPTV 포함) 2순위 :  산책 및 걷기 3순위 :  잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함) 4순위 :  목욕/사우나/찜질방 5순위 :  친구만남/이성교제/미팅/소개팅 '

In [14]:
# 사용자가 입력한 문장
prompt = f'''너는 자연어처리를 통해 사용자에게 정보를 제공하는 챗봇이야 주어진 자료를 가지고 선호도가 높은 정보를 '사용자님과 비슷한 유형의 사람들은 ~에 관심이 높습니다.'로 답해줘 

자료:
{document}
'''

response = model.generate_content(prompt)
answer = response.candidates[0].content.parts[0].text
print(answer)

사용자님과 비슷한 유형의 사람들은 **모바일 컨텐츠, 동영상, VOD 시청, OTT 시청**에 관심이 높습니다. 

